In [1]:
## Importing libraries

import torch
import torch.optim as optim
import torch.nn as nn
import os
from torch.utils.data import Dataset, DataLoader , random_split
from torchvision import transforms , datasets
from PIL import Image
import pandas as pd
import re
import random
import shutil
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
import numpy as np



/home/jaskaran/.conda/envs/training/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## checking if cuda is available
if torch.cuda.is_available():
    print("CUDA is available!")
else:
    print("CUDA is not availabe")


CUDA is available!


In [3]:
## Custom class made for dataset training
class GeneticMatrixDataset(Dataset):
    def __init__(self,root_folder, transform =None):
        self.root_folder = root_folder
        self.transform = transform
        self.class_labels = sorted(os.listdir(root_folder))
        self.class_to_idx = {label: idx for idx ,label in enumerate(self.class_labels)}
        self.file_list = self._build_file_list()
    
    def _build_file_list(self):
        file_list = []
        for class_label in self.class_labels:
            class_path = os.path.join(self.root_folder,class_label)
            class_image = [os.path.join(class_path,image_path) for image_path in os.listdir(class_path)]
            file_list.extend(class_image)
        return file_list
    
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self,index):
        file_path = self.file_list[index]
        array = np.loadtxt(file_path)
        mean = np.mean(array)
        std = np.std(array)
        array = (array - mean) / std ##normalized array
        class_label = os.path.basename(os.path.dirname(file_path))
        label = self.class_to_idx[class_label]
        find = re.match(r".+?[_].+?[_](.+?)[.][t][x][t]",file_path)
        array = torch.from_numpy(array)
        return {'array': array, 'label': label, "coefficient ":find[1]}

In [4]:


data = GeneticMatrixDataset("/home/jaskaran/data_matrix") ## loading the data
generator = torch.Generator().manual_seed(46) ## seeding the process
length_data = len(data)
training_size = int(0.8*length_data) ## 80 percent as training size
testing_size = length_data - training_size ## 20 oercent of data as testing size

train_data , test_data = random_split(data,[training_size, testing_size],generator=generator)


training_dataloader= DataLoader(train_data,batch_size=32,shuffle=True, drop_last=True) ## Intialising the dataloader
testing_dataloader = DataLoader(test_data,batch_size=32,shuffle=True, drop_last=True)



In [6]:
## CNN architecture used for the training
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, 
                               out_channels=32,
                               kernel_size=(15,9), 
                               stride= 1,
                               padding =15)
        self.relu = nn.ReLU(inplace=True)
        self.softmax = nn.Softmax()
        self.maxpool = nn.MaxPool2d(kernel_size =3)
        self.fc1 = nn.Linear(424320,256)
        self.fc2 = nn.Linear(256,128)
        self.fc3 = nn.Linear(128,2)
        self.flatten = nn.Flatten(start_dim=1)

        self.dropout = nn.Dropout(p=0.2)
    
    def forward(self,x):
        x = x.float()

        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.flatten(x)

        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)

        x = self.fc2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.fc3(x) 
        return x

In [7]:
# Set seed for CPU operations
torch.manual_seed(45)

# Check if CUDA is available
if torch.cuda.is_available():
    # Set seed for CUDA operations
    torch.cuda.manual_seed(45)
    torch.cuda.manual_seed_all(45)  
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
model.to("cuda:0")
num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    
    for batch in training_dataloader:
        matrix = batch['array']
        matrix = matrix.unsqueeze(1)
        matrix = matrix.to("cuda:0")
        labels = batch['label'].to("cuda:0")  # Ensure labels are also moved to GPU
        optimizer.zero_grad()
        outputs = model(matrix)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for batch in testing_dataloader:
            matrix = batch['array']
            matrix = matrix.unsqueeze(1)
            matrix = matrix.to("cuda:0")
            labels = batch['label'].to("cuda:0")
            output = model(matrix)
            _, predicted = torch.max(output.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
    accuracy = correct / total 
    print(f'Epoch {epoch + 1}/{num_epochs},loss : {loss:.2f} Accuracy: {accuracy:.2%}')

print("Model is trained")

Epoch 1/20,loss : 0.38 Accuracy: 97.33%


Epoch 2/20,loss : 0.10 Accuracy: 97.73%


Epoch 3/20,loss : 0.24 Accuracy: 97.68%


Epoch 4/20,loss : 0.07 Accuracy: 96.67%


Epoch 5/20,loss : 0.27 Accuracy: 97.08%


...

Epoch 16/20,loss : 0.02 Accuracy: 97.88%


Epoch 17/20,loss : 0.39 Accuracy: 97.98%


Epoch 18/20,loss : 0.28 Accuracy: 97.18%


Epoch 19/20,loss : 0.03 Accuracy: 97.93%


Epoch 20/20,loss : 0.10 Accuracy: 97.88%
Model is trained


In [8]:
## Save the model dict
torch.save(model.state_dict(), f'model_state_dict_matrix_lr_0.0001_15*9_dropout_0.2.pth')